In [1]:
from tqdm import tqdm
import pickle

# facts = pickle.load(open('wikidata_text_facts.txt', 'rb'))

# facts = pickle.load(open('text_facts_v2.pkl', 'rb'))
facts = readFactsFromFile('data/temporal_big/full.txt')

FileNotFoundError: [Errno 2] No such file or directory: 'text_facts_v2.pkl'

In [1]:
def getEntsFromFacts(facts):
    e = set()
    for f in facts:
        e.add(f[0])
        e.add(f[2])
    return e

def readFile(filename):
    f = open(filename, 'r')
    lines = []
    for line in f:
        line = line.strip()
        if line != '':
            lines.append(line)
    f.close()
    return lines

def getRelsFromFacts(facts):
    r = set()
    for f in facts:
        r.add(f[1])
    return r

def filterByRelation(facts, rel, max=50):
    filtered = []
    count = 0
    for f in facts:
        if f[1] == rel:
            filtered.append(f)
            count += 1
        if max < 0:
            continue
        if count >= max:
            break
    return filtered

def filterByHead(facts, head, max=50):
    filtered = []
    count = 0
    for f in facts:
        if f[0] == head:
            filtered.append(f)
            count += 1
        if max < 0:
            continue
        if count >= max:
            break
    return filtered

def filterByTail(facts, tail, max=50):
    filtered = []
    count = 0
    for f in facts:
        if f[2] == tail:
            filtered.append(f)
            count += 1
        if max < 0:
            continue
        if count >= max:
            break
    return filtered

def filterByEntity(facts, ent, max=50):
    filtered = []
    count = 0
    for f in facts:
        if f[0] == ent or f[2] == ent:
            filtered.append(f)
            count += 1
        if max < 0:
            continue
        if count >= max:
            break
    return filtered

def printFact(f):
    s = "{head}, {rel}, {tail}, {t1}, {t2}"
    head=f[0]
    rel = f[1]
    tail = f[2]
    t1 = f[3][1:5]
    t2 = f[4][1:5]
    print(s.format(head=head, rel=rel, tail=tail, t1=t1, t2=t2))
    
def printFacts(facts):
    for f in facts:
        printFact(f)
        
def isEntityInFact(e, fact):
    if fact[0] == e or fact[2] == e:
        return True
    else:
        return False
    
        
def writeFactsToFile(filename, facts):
    f = open(filename, 'w')
    for fact in facts:
        line = '\t'.join(fact)
        f.write(line + '\n')
    f.close()
    
def readFactsFromFile(filename):
    f = open(filename, 'r')
    facts = []
    for line in f:
        fact = line.strip().split('\t')
        facts.append(fact)
    f.close()
    return facts

def createWikidataIdentifierToTextDict(entities, relations):
    ent_rel_both = {}
    ent_rel_both['Q'] = entities
    ent_rel_both['P'] = relations
    wikidata_id2name = {}
    count = 0
    for prefix, lines in ent_rel_both.items():
        for line in lines:
            try:
                line = line.split('\t')
                id = prefix + line[1] #different from one above
                name = line[2]
                wikidata_id2name[id] = name
            except:
                id = prefix + line[1]
                name = ''
                wikidata_id2name[id] = name
    return wikidata_id2name

def convertFactToText(fact, wikidata_identifier_to_text):
    f = fact.copy()
    for i in range(3):
        f[i] = wikidata_identifier_to_text[fact[i]]
    return f

In [4]:
facts[0]

['Q25559009', 'P39', 'Q41582555', '1847', '1852']

In [3]:
# facts = readFactsFromFile('data/temporal_small/full.txt')
facts = readFactsFromFile('data/temporal_big/full.txt')

In [5]:
r = getRelsFromFacts(facts)
e = getEntsFromFacts(facts)

In [6]:
len(facts)

328635

In [7]:
len(e)

125726

In [8]:
e_count_dict = {}
r_count_dict = {}
head_count = {}
tail_count = {}
for ent in e:
    e_count_dict[ent] = 0
    head_count[ent] = 0
    tail_count[ent] = 0
for rel in r:
    r_count_dict[rel] = 0
for f in facts:
    e1 = f[0]
    e2 = f[2]
    rel = f[1]
    e_count_dict[e1] += 1
    e_count_dict[e2] += 1
    r_count_dict[rel] += 1
    head_count[e1] += 1
    tail_count[e2] += 1

In [9]:
x = r_count_dict
sorted_r = {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}

x = e_count_dict
sorted_e = {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}

x = head_count
sorted_head = {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}

x = tail_count
sorted_tail = {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
for key, value in sorted_r.items():
    print(key, '|', value, 'facts', round(value/len(facts)*100, 2), '%' )

In [10]:
#rel = 'position held'
rel = 'P39'
sportsFacts = filterByRelation(facts, rel, -1)

In [11]:
len(sportsFacts)

78380

In [12]:
# position_held = filterByRelation(facts, 'position held', -1)
position_held = filterByRelation(facts, 'P39', -1)
len(position_held)

78380

In [13]:
filtered_facts = []
for f in position_held:
    tail = f[2]
    if tail != tail.lower():
        filtered_facts.append(f)

In [14]:
len(filtered_facts)

78380

In [15]:
def splitTimeNoNewTimeStamps(facts, timestamps):
    # don't introduce new timestamps!
    new_facts = []
    for f in facts:
        e1 = f[0]
        r = f[1]
        e2 = f[2]
        t1 = int(f[3])
        t2 = int(f[4])
        for t in range(t1, t2+1):
            if t in timestamps:
                nf = [e1, r, e2, t]
                new_facts.append(nf)
    return new_facts

def splitTime(facts):
    # don't introduce new timestamps!
    new_facts = []
    for f in facts:
        e1 = f[0]
        r = f[1]
        e2 = f[2]
        t1 = int(f[3])
        t2 = int(f[4])
        for t in range(t1, t2+1):
            nf = [e1, r, e2, t]
            new_facts.append(nf)
    return new_facts



In [16]:
timestamps = set()
for f in facts:
    timestamps.add(int(f[-2]))
    timestamps.add(int(f[-1]))
print(len(timestamps))


1643


In [17]:
x = splitTime(filtered_facts)

In [18]:
len(x)

422446

In [19]:
len(filtered_facts)

78380

In [20]:
sportsFactsSingle = x

In [21]:
sportsFactsSingle[0]

['Q25559009', 'P39', 'Q41582555', 1847]

In [22]:
# simple questions, time answer
def genTimeSimple1(facts, base_fact):
    head = base_fact[0]
    tail = base_fact[2]
    template = "When did {head} hold the position of {tail}?"
    answers = set()
    for f in facts:
        # (s, r, o, T)
        if f[0] == head and f[2] == tail:
            answers.add(f[3])
    question = template.format(head=head, tail=tail)
    answer_type = 'time'
    entities = set([head, tail])
    times = set()
    relations = set(['P39'])
    output = {'question': question,
             'answers': answers,
             'answer_type': answer_type,
             'template': template,
             'entities': entities,
             'times': times,
             'relations': relations}
    return output

# complex questions, time answer
def genTimeComplex1(facts, head, tail, first=True):
    template = "When was the {adj} time that {head} was the {tail}?"
    minTime = 9999
    maxTime = -1
    answers = set()
    for f in facts:
        if f[0] == head and f[2] == tail:
            time = f[3]
            if time < minTime:
                minTime = time
            if time > maxTime:
                maxTime = time
    if first==True:
        question = template.format(head=head, tail=tail, adj='first')
        answers.add(minTime)
    else:
        question = template.format(head=head, tail=tail, adj='last')
        answers.add(maxTime)
    answer_type = 'time'
    entities = set([head, tail])
    times = set()
    relations = set(['P39'])
    output = {'question': question,
             'answers': answers,
             'answer_type': answer_type,
             'template': template,
             'entities': entities,
             'times': times,
             'relations': relations}
    return output
        
def genTimeComplex2(facts, tail, first=True):
    template = "When did the {adj} {tail} come to power?" # ERROR- semantically correct only for president, etc
    minTime = 9999
    maxTime = -1
    answers = set()
    for f in facts:
        if f[2] == tail:
            time = f[3]
            if time < minTime:
                minTime = time
            if time > maxTime:
                maxTime = time
    if first==True:
        question = template.format(tail=tail, adj='first')
        answers.add(minTime)
    else:
        question = template.format(tail=tail, adj='last')
        answers.add(maxTime)
    answer_type = 'time'
    entities = set([tail])
    times = set()
    relations = set(['P39'])
    output = {'question': question,
             'answers': answers,
             'answer_type': answer_type,
             'template': template,
             'entities': entities,
             'times': times,
             'relations': relations}
    return output

# simple questions, entity answer
def genEntitySimple1(facts, tail, time):
    template = "Who was the {tail} in {time}?"
    answers = set()
    for f in facts:
        if f[2] == tail and f[3] == time:
            answers.add(f[0])
    question = template.format(tail=tail, time=time)
    answer_type = 'entity'
    entities = set([tail])
    times = set([time])
    relations = set(['P39'])
    output = {'question': question,
             'answers': answers,
             'answer_type': answer_type,
             'template': template,
             'entities': entities,
             'times': times,
             'relations': relations}
    return output

def genEntitySimple2(facts, tail, time1, time2):
    template = "Who was the {tail} from {time1} to {time2}?"
    answers = set()
    for f in facts:
        if f[2] == head and f[3] == time: # ERROR f[2]>=head and f[3]<=tail
            answers.add(f[2])
    question = template.format(head=head, time=time)
    answer_type = 'entity'
    entities = set([tail])
    times = set([time1, time2])
    relations = set(['P39'])
    output = {'question': question,
             'answers': answers,
             'answer_type': answer_type,
             'template': template,
             'entities': entities,
             'times': times,
             'relations': relations}
    return output


# complex question, entity answer
def genEntityComplex1(facts, tail, first=True):
    # first/last
    template = "Who was the {adj} {tail}?"
    answers = set()
    if first==True:
        x = getFactWithMinimumTime(facts, head ='', tail=tail)
        time=x[-1]
        for f in facts:
            if f[-1]==time and f[-2]==tail:
                answers.add(f[0])
        question = template.format(tail=x[2], adj='first')
    else:
        x = getFactWithMaximumTime(facts, head ='', tail=tail)
        time=x[-1]
        for f in facts:
            if f[-1]==time and f[-2]==tail:
                answers.add(f[0])
        question = template.format(tail=x[2], adj='last')
    answer_type = 'entity'
    entities = set([tail])
    times = set()
    relations = set(['P39'])
    output = {'question': question,
             'answers': answers,
             'answer_type': answer_type,
             'template': template,
             'entities': entities,
             'times': times,
             'relations': relations}
    return output

def getFactWithMaximumTime(facts, head = '', tail = ''):
    maxTime = -1
    fact = facts[0]
    for f in facts:
        time = f[-1]
        if head != '':
            if f[0] != head:
                continue
        if tail != '':
            if f[2] != tail:
                continue
        if time > maxTime:
            maxTime = time
            fact = f
    return fact

def getFactWithMinimumTime(facts, head = '', tail = ''):
    minTime = 9999
    fact = facts[0]
    for f in facts:
        time = f[-1]
        if head != '':
            if f[0] != head:
                continue
        if tail != '':
            if f[2] != tail:
                continue
        if time < minTime:
            minTime = time
            fact = f
    return fact


def genEntityComplex2(facts, head, tail, after=True):
    # before/after
    template = "Who was the {tail} {type} {head}?"
    # before: find argmin(T) (head, r, tail, T)
    # then, find argmax(T')(head, r, tail', T'), T' < T and head' != head
    # tail' is the answer
    if after == False:
        base_fact = getFactWithMinimumTime(facts, head = head, tail = tail)
        time = base_fact[-1]
        maxTime = -1
        for f in facts:
            time2 = f[-1]
            if time2 >= time:
                continue
            if f[0] == head:
                continue
            if f[2] != tail:
                continue
            if time2 > maxTime:
                maxTime = time2
        answers = set()
        for f in facts:
            time2 = f[-1]
            if f[0] == head:
                continue
            if f[2] != tail:
                continue
            if time2 == maxTime:
                answers.add(f[0])
        question = template.format(head=head, type="before", tail=tail)
    else:
        #after: find argmax(T) (head, r, tail, T)
        # then, find argmin(T') (head, r, tail', T'), T' > T and head' != head
        # tail' is the answer
        base_fact = getFactWithMaximumTime(facts, head = head, tail = tail)
        time = base_fact[-1]
        minTime = 9999
        for f in facts:
            time2 = f[-1]
            if time2 <= time:
                continue
            if f[0] == head:
                continue
            if f[2] != tail:
                continue
            if time2 < minTime:
                minTime = time2
        answers = set()
        for f in facts:
            time2 = f[-1]
            if f[0] == head:
                continue
            if f[2] != tail:
                continue
            if time2 == minTime:
                answers.add(f[0])
        question = template.format(head=head, type="after", tail=tail)
    answer_type = 'entity'
    entities = set([head, tail])
    times = set()
    relations = set(['P39'])
    if len(answers)==0:
        return None
    output = {'question': question,
             'answers': answers,
             'answer_type': answer_type,
             'template': template,
             'entities': entities,
             'times': times,
             'relations': relations}
    return output
    
# def genEntityComplex3(facts, head1, head2, tail):
#     template = "Which team did {head1} play for when {head2} was playing for "


def genEntityComplex3(facts, head, tail, tail2):
    template = "Who held the position of {tail2} when {head} was the {tail}?"
    # first get all time instances when head played for tail
    # then get all heads, where tail' = tail and time' in T
    valid_times = set()
    for f in facts:
        if f[0] == head and f[2] == tail:
            valid_times.add(f[-1])
    answers = set()
    for f in facts:
        if f[2] == tail2 and f[-1] in valid_times:
            answers.add(f[0])
    question = template.format(head=head, tail=tail, tail2=tail2)
    answer_type = 'entity'
    entities = set([head, tail, tail2])
    times = set()
    relations = set(['P39'])
    if len(answers)==0:
        return None
    output = {'question': question,
             'answers': answers,
             'answer_type': answer_type,
             'template': template,
             'entities': entities,
             'times': times,
             'relations': relations}
    return output
    
def genEventQuestion(facts, events_fact, tail):
    event_head = events_fact[0]
    template = "Who held the position of {tail} during {event_head}?"
    valid_times = set()
    start_time = int(events_fact[-2])
    end_time = int(events_fact[-1])
    for i in range(start_time, end_time + 1):
        valid_times.add(i)
    answers = set()
    for f in facts:
        if f[2] == tail and f[-1] in valid_times:
            answers.add(f[0])
    question = template.format(event_head=event_head, tail=tail)
    answer_type = 'entity'
    entities = set([tail, event_head])
    times = set()
    relations = set(['P39'])
    if len(answers)==0:
        return None
    output = {'question': question,
             'answers': answers,
             'answer_type': answer_type,
             'template': template,
             'entities': entities,
             'times': times,
             'relations': relations}
    return output

def genEventQuestion2(facts, events_fact, tail, after=True):
    event_head = events_fact[0]
    template = "Who held the position of {tail} {type} {event_head}?"
    answers = set()
    # event happened at some time
    # choose all times before/after that time
    # choose fact with tail=tail with max/min time in those valid times
    # head of that fact is the answer
    if after == False:
        # before
        # get upper limit time
        upper_limit = int(events_fact[-2])
        maxTime = -1
        # now get the fact with time < upper_limit
        for f in facts:
            time = f[-1]
            if time >= upper_limit:
                continue
            if tail != f[2]:
                continue
            if time > maxTime:
                maxTime = time
        answers = set()
        for f in facts:
            time = f[-1]
            if tail != f[2] or head==f[0]:
                continue
            if time == maxTime:
                answers.add(f[0])
        question = template.format(event_head=event_head, type="before", tail=tail)
    else:
        # after
        # get lower limit time
        lower_limit = int(events_fact[-1])
        minTime = 99999
        # now get the fact with time > lower_limit
        for f in facts:
            time = f[-1]
            if time <= lower_limit:
                continue
            if tail != f[2]:
                continue
            if time < minTime:
                minTime = time
        answers = set()
        for f in facts:
            time = f[-1]
            if tail != f[2] or head==f[0]:
                continue
            if time == minTime:
                answers.add(f[0])
        question = template.format(event_head=event_head, type="after", tail=tail)
    answer_type = 'entity'
    entities = set([tail, event_head])
    times = set()
    relations = set(['P39'])
    if len(answers)==0:
        return None
    output = {'question': question,
             'answers': answers,
             'answer_type': answer_type,
             'template': template,
             'entities': entities,
             'times': times,
             'relations': relations}
    return output

def areFactsSame(f1, f2):
    flag = True
    for i in range(3):
        if f1[i] != f2[i]:
            flag = False
            break
    return flag

def toQuestionJSON(question, answers, template, wikidata_identifier_to_text):
    output = {}
    output['']

In [23]:
entities = readFile('fb_wikidata/entities')
relations = readFile('fb_wikidata/predicates')

In [24]:
wikidata_identifier_to_text = createWikidataIdentifierToTextDict(entities, relations)
text_to_wikidata_identifier = {v: k for k, v in wikidata_identifier_to_text.items()}

In [39]:
# positionFactsSingle = [convertFactToText(f, wikidata_identifier_to_text) for f in sportsFactsSingle]
positionFactsSingle = sportsFactsSingle

In [ ]:
posi

In [26]:
id = 600
f = positionFactsSingle[id]
genTimeSimple1(positionFactsSingle, f)

{'question': 'When did Q324970 hold the position of Q2047357?',
 'answers': {1935,
  1936,
  1937,
  1938,
  1939,
  1940,
  1941,
  1942,
  1943,
  1944,
  1945,
  1946},
 'answer_type': 'time',
 'template': 'When did {head} hold the position of {tail}?',
 'entities': {'Q2047357', 'Q324970'},
 'times': set(),
 'relations': {'P39'}}

In [27]:
genTimeComplex2(positionFactsSingle, f[2], first=True)

{'question': 'When did the first Q2047357 come to power?',
 'answers': {1813},
 'answer_type': 'time',
 'template': 'When did the {adj} {tail} come to power?',
 'entities': {'Q2047357'},
 'times': set(),
 'relations': {'P39'}}

In [28]:
genTimeComplex1(positionFactsSingle, f[0], f[2], first=True)

{'question': 'When was the first time that Q324970 was the Q2047357?',
 'answers': {1935},
 'answer_type': 'time',
 'template': 'When was the {adj} time that {head} was the {tail}?',
 'entities': {'Q2047357', 'Q324970'},
 'times': set(),
 'relations': {'P39'}}

In [29]:
genEntitySimple1(positionFactsSingle, f[2], f[-1])

{'question': 'Who was the Q2047357 in 1946?',
 'answers': {'Q324970'},
 'answer_type': 'entity',
 'template': 'Who was the {tail} in {time}?',
 'entities': {'Q2047357'},
 'times': {1946},
 'relations': {'P39'}}

In [30]:
genEntityComplex1(positionFactsSingle, f[2], first=True)

{'question': 'Who was the first Q2047357?',
 'answers': {'Q26286363'},
 'answer_type': 'entity',
 'template': 'Who was the {adj} {tail}?',
 'entities': {'Q2047357'},
 'times': set(),
 'relations': {'P39'}}

In [31]:
genEntityComplex2(positionFactsSingle, f[0], f[2], after=True)

{'question': 'Who was the Q2047357 after Q324970?',
 'answers': {'Q1790063'},
 'answer_type': 'entity',
 'template': 'Who was the {tail} {type} {head}?',
 'entities': {'Q2047357', 'Q324970'},
 'times': set(),
 'relations': {'P39'}}

In [32]:
genEntityComplex3(positionFactsSingle, f[0], f[2], 'Prime minister of India')

{'question': 'Who held the position of Prime minister of India when Q324970 was the Q2047357?',
 'answers': set(),
 'answer_type': 'entity',
 'template': 'Who held the position of {tail2} when {head} was the {tail}?',
 'entities': {'Prime minister of India', 'Q2047357', 'Q324970'},
 'times': set(),
 'relations': {'P39'}}

In [33]:
# with (open("temporal_events_as_text_facts.pkl", "rb")) as openfile:
#     events_facts = pickle.load(openfile)
# events_facts = readFactsFromFile('facts_temporal_event.txt')

# events facts are already there in facts

In [25]:
def getEventsFactsFromFacts(facts):
    events_facts = []
    for f in facts:
        rel = f[1]
        tail = f[2]
        if rel == 'P793' and tail == 'Q1190554': # significant event and occurrence
#         if rel == 'P39' and tail == 'Q1190554': # significant event and occurrence
            events_facts.append(f)
    return events_facts
events_facts = getEventsFactsFromFacts(facts)


In [46]:
# len(events_facts)
import pickle as pkl
def get_facts_from_ent(entname):
    idx=name2ent[entname]
    facts_=[]
    for fact in events_facts:
        if (fact[0]==idx or fact[2]==idx):
            fact=(ent2name[fact[0]],rel2name[fact[1]],ent2name[fact[2]],fact[3],fact[4])
            facts_.append(fact)
            print(fact)
    return facts_
def openFileAsDict(filename):
    f = open(filename, 'r')
    out = {}
    for line in f:
        line = line[:-1].split('\t') # can't strip() since name can be whitespace
        out[line[0]] = line[1]
    return out
ent2name = openFileAsDict( '../Temporal_KGQA/data/wikidata_big/kg/wd_id2entity_text.txt')
rel2name = openFileAsDict( '../Temporal_KGQA/data/wikidata_big/kg/wd_id2relation_text.txt')
name2ent={i:j for (j,i) in ent2name.items()}
name2rel={i:j for (j,i) in rel2name.items()}

ent2id=pkl.load(open("../Temporal_KGQA/data/wikidata_big/kg/tkbc_processed_data/wikidata_big/ent_id","rb"))
rel2id=pkl.load(open("../Temporal_KGQA/data/wikidata_big/kg/tkbc_processed_data/wikidata_big/rel_id","rb"))
id2ent={i:j for (j,i) in ent2id.items()}
id2rel={i:j for (j,i) in rel2id.items()}
id2relname={i:rel2name[id2rel[i]] for i in id2rel}
id2entname={i:ent2name[id2ent[i]] for i in id2ent}


In [32]:
name2ent["Member of the Victorian Legislative Assembly"]

'Q18534408'

In [49]:
facts_reqd=get_facts_from_ent("2012 Summer Olympics")

('2012 Summer Olympics', 'significant event', 'occurrence', '2012', '2012')


In [36]:
genEventQuestion(positionFactsSingle, events_facts[0], 'Q11696')

{'question': 'Who held the position of Q11696 during Q362?',
 'answers': {'Q11613', 'Q8007'},
 'answer_type': 'entity',
 'template': 'Who held the position of {tail} during {event_head}?',
 'entities': {'Q11696', 'Q362'},
 'times': set(),
 'relations': {'P39'}}

In [54]:
print(facts_reqd)
genEventQuestion2(positionFactsSingle, facts_reqd[0], 'Q18534408', after=False)

[('2012 Summer Olympics', 'significant event', 'occurrence', '2012', '2012')]
Rupert Hamer position held Member of the Victorian Legislative Assembly 1971
Rupert Hamer position held Member of the Victorian Legislative Assembly 1972
Rupert Hamer position held Member of the Victorian Legislative Assembly 1973
Rupert Hamer position held Member of the Victorian Legislative Assembly 1974
Rupert Hamer position held Member of the Victorian Legislative Assembly 1975
Rupert Hamer position held Member of the Victorian Legislative Assembly 1976
Rupert Hamer position held Member of the Victorian Legislative Assembly 1977
Rupert Hamer position held Member of the Victorian Legislative Assembly 1978
Rupert Hamer position held Member of the Victorian Legislative Assembly 1979
Rupert Hamer position held Member of the Victorian Legislative Assembly 1980
Rupert Hamer position held Member of the Victorian Legislative Assembly 1981
James Vinton Smith position held Member of the Victorian Legislative Assemb

{'question': 'Who held the position of Q18534408 before 2012 Summer Olympics?',
 'answers': {'Q6939485'},
 'answer_type': 'entity',
 'template': 'Who held the position of {tail} {type} {event_head}?',
 'entities': {'2012 Summer Olympics', 'Q18534408'},
 'times': set(),
 'relations': {'P39'}}

In [38]:
wikidata_identifier_to_text['Q11696']

'President of the United States'

In [39]:
id = 200
f = positionFactsSingle[id]
print(genTimeSimple1(positionFactsSingle, f))
print(genTimeComplex2(positionFactsSingle, f[2], first=True))
print(genTimeComplex1(positionFactsSingle, f[0], f[2], first=True))
print(genEntitySimple1(positionFactsSingle, f[2], f[-1]))
print(genEntityComplex1(positionFactsSingle, f[2], first=True))
print(genEntityComplex2(positionFactsSingle, f[0], f[2], after=True))
print(genEntityComplex3(positionFactsSingle, f[0], f[2], 'Prime minister of India'))
print(genEventQuestion(positionFactsSingle, events_facts[0], 'President of the United States'))

{'question': 'When did Q7173773 hold the position of Q20165902?', 'answers': {1968, 1965, 1966, 1967}, 'answer_type': 'time', 'template': 'When did {head} hold the position of {tail}?', 'entities': {'Q7173773', 'Q20165902'}, 'times': set(), 'relations': {'P39'}}
{'question': 'When did the first Q20165902 come to power?', 'answers': {1890}, 'answer_type': 'time', 'template': 'When did the {adj} {tail} come to power?', 'entities': {'Q20165902'}, 'times': set(), 'relations': {'P39'}}
{'question': 'When was the first time that Q7173773 was the Q20165902?', 'answers': {1965}, 'answer_type': 'time', 'template': 'When was the {adj} time that {head} was the {tail}?', 'entities': {'Q7173773', 'Q20165902'}, 'times': set(), 'relations': {'P39'}}
{'question': 'Who was the Q20165902 in 1966?', 'answers': {'Q7173773'}, 'answer_type': 'entity', 'template': 'Who was the {tail} in {time}?', 'entities': {'Q20165902'}, 'times': {1966}, 'relations': {'P39'}}
{'question': 'Who was the first Q20165902?', 'a

In [40]:
import random
def makeQuestions(positionFactsSingle, events_facts, split_facts, f):
    questions = []
    questions.append(genTimeSimple1(positionFactsSingle, f))
    questions.append(genTimeComplex2(positionFactsSingle, f[2], first=random.choice([True, False])))
    questions.append(genTimeComplex1(positionFactsSingle, f[0], f[2], first=random.choice([True, False])))
    questions.append(genEntitySimple1(positionFactsSingle, f[2], f[-1]))
    questions.append(genEntityComplex1(positionFactsSingle, f[2], first=random.choice([True, False])))
    questions.append(genEntityComplex2(positionFactsSingle, f[0], f[2], after=random.choice([True, False])))
    questions.append(genEntityComplex3(positionFactsSingle, f[0], f[2], random.choice(split_facts)[2]))
    # using only the first 10 events facts ?.?
    # because we want world war II to be there
    # which is the first event fact lol xD
    questions.append(genEventQuestion(positionFactsSingle, random.choice(events_facts[:10]), f[2]))
    questions.append(genEventQuestion2(positionFactsSingle, random.choice(events_facts[:10]), f[2], after=random.choice([True, False])))
    return questions


In [41]:
train_ents = set(pickle.load(open('train_ents.pickle', 'rb')))
test_ents = set(pickle.load(open('test_ents.pickle', 'rb')))

len(train_ents.intersection(test_ents))

0

In [42]:
import pickle
dataset_split = 'train'
split_entities = set(pickle.load(open(dataset_split + '_ents.pickle', 'rb')))
split_facts = []
my_facts = positionFactsSingle
for f in my_facts:
    if f[0] in split_entities and f[2] in split_entities:
        split_facts.append(f)
        
len(split_facts)

232721

In [43]:
# how to get number of questions?
# 1. set total questions to 300k
# 2. get number of facts with this relation
# 3. get how many questions u want of this relation as fraction
# 4. choose 10% of that number for test, 90% for train
max_dataset_questions = 300000
relation_name = my_facts[0][1]
num_relation_facts = len(filterByRelation(facts, relation_name, -1))
# fraction of questions need to be decided based on fraction of question relation facts
# not all facts!
question_relation_list = ['P39', 'P166', 'P108', 'P54', 'P26']
num_all_question_relation_facts = sum([len(filterByRelation(facts, x, -1)) for x in question_relation_list])
num_questions_for_this_relation = int(max_dataset_questions * num_relation_facts/num_all_question_relation_facts)
split_ratios = {'test': 0.1, 'train': 0.9}
num_questions = int(split_ratios[dataset_split] * num_questions_for_this_relation)
num_questions

91082

In [44]:
num_all_question_relation_facts

232343

In [45]:
import random
data = []
# num_questions has been calculated above
# num_questions = 100000

events_facts_small = events_facts[:10]
questions_set = set()

pbar = tqdm(range(len(split_facts)))
random.shuffle(split_facts)
for i in range(len(split_facts)):
#     f = random.choice(positionFactsSingle)
    f = split_facts[i]
    pbar.set_description("Num questions %d, i %d" % (len(data), i))
    questions = makeQuestions(my_facts, events_facts_small, split_facts, f)
    for q in questions:
        if len(q['answers']) > 0 and q['question'] not in questions_set:
            data.append(q)
            questions_set.add(q['question'])
    if len(data) >= num_questions:
        break

Num questions 36334, i 9061:   0%|          | 0/232721 [50:48<?, ?it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Num questions 53752, i 14889:   0%|          | 0/232721 [1:45:28<?, ?it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Num questions 68019, i 20219:   0%|          | 0/232721 [2:37:16<?, ?it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.


In [51]:
len(data)

91082

In [52]:
def getQuestionTypeDistribution(data):
    type_dict = {}
    for d in data:
        template = d['template']
        if template not in type_dict:
            type_dict[template] = 1
        else:
            type_dict[template] += 1
    return type_dict

In [53]:
getQuestionTypeDistribution(data)

{'When did {head} hold the position of {tail}?': 16632,
 'When did the {adj} {tail} come to power?': 1668,
 'When was the {adj} time that {head} was the {tail}?': 21131,
 'Who was the {tail} in {time}?': 10665,
 'Who was the {adj} {tail}?': 1687,
 'Who was the {tail} {type} {head}?': 14354,
 'Who held the position of {tail2} when {head} was the {tail}?': 18034,
 'Who held the position of {tail} during {event_head}?': 1999,
 'Who held the position of {tail} {type} {event_head}?': 4912}

In [54]:
import pickle
# filename = 'data/questions/questions_position_held_big.pickle'
filename = 'data/questions/{split}_questions_position_held_big.pickle'.format(
            split=dataset_split)
pickle.dump(data, open(filename, "wb"))

In [ ]:
# y = readQuestions('questions_position_held.txt')
# y[50:100]

In [ ]:
data[50:100]

In [50]:
len(data)

91082

In [ ]:
x = set()
for d in data:
    x.add(d['question'])
len(x)